In [1]:
import torch
from torch import nn

In [4]:
class ConvBNReLU(nn.Module):
    def __init__(self, inplanes, outplanes, kernel_size, stride, groups):
        super(ConvBNReLU, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv = None
        self.bn = nn.BatchNorm2d()
        self.act = None
        
    def forward(self,x):
        
        x1 = self.conv(x)
        x1 = self.bn(x1)
        x1 = self.act(x1)
        
        return x1

In [2]:
class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, shortcut):
        super(InvertedResidual, self).__init__()
        
        self.shortcut = shortcut # True, False
        
        hidden_dim = int(round(inp * expand_ratio))
        
        self.expand_pw = ConvBNReLU()
        self.depthwise = ConvBNReLU()
        self.squeeze_pw = ConvBNReLU()
        
    def forward(self, x):
        x1 = self.expand_pw(x)
        x1 = self.depthwise(x)
        x1 = self.squeeze_pw(x)
        
        if self.shortcut:
            out = x + x1
        else:
            out = x1
            
        return out

In [ ]:
class LGMobileNetV2(nn.Module):
    def __init__(self, num_classes):
        super(LGMobileNetV2, self).__init__()
        
        # input : 3 x 150 x 150
        self.stem = ConvBNReLU(inplanes=3, outplanes=16, kernel_size=3, stride=2) # 16 x 75 x 75
        self.block1 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 16 x 75 x 75
        self.block2 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 32 x 38 x 38
        self.block3 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 64 x 19 x 19
        self.block4 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 64 x 19 x 19
        self.block5 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 128 x 10 x 10
        self.block6 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 128 x 5 x 5
        self.block7 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 128 x 5 x 5
        self.block8 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 128 x 3 x 3
        self.pool = nn.AvgPool2d(kernel_size=3)
        
        self.classifier = nn.Linear(128,2)
        
    def forward(self, x):
        x1 = self.stem(x)
        
        x1 = self.block1(x1)
        x1 = self.block2(x1)
        x1 = self.block3(x1)
        x1 = self.block4(x1)
        x1 = self.block5(x1)
        x1 = self.block6(x1)
        x1 = self.block7(x1)
        x1 = self.block8(x1)
        
        x1 = x1.reshape(x.shape[0], -1) # 2차원으로 바꾸는 코드
        x1 = self.classifier(x1)
        
        return x1

---